In [5]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import gc

In [8]:
print('Loading data ...')

train = pd.read_csv('train_2016_v2.csv')
prop = pd.read_csv('properties_2016.csv')

Loading data ...


/Users/yan/anaconda/envs/py35/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [13]:
prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.369141,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.570312,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.169922,NaN,NaN,NaN


In [9]:
for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')

In [10]:
df_train.shape

(90275, 60)

In [14]:
df_train.head()

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.879883,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.019531,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.480469,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.739990,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.959961,NaN,NaN,6.059042e+13


In [15]:
x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

(90275, 55) (90275,)


In [16]:
train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

524

In [17]:
split = 90000
x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
x_train = x_train.values.astype(np.float32, copy=False)
x_valid = x_valid.values.astype(np.float32, copy=False)

In [18]:
d_train = lgb.Dataset(x_train, label=y_train)
d_valid = lgb.Dataset(x_valid, label=y_valid)

params = {}
params['learning_rate'] = 0.002
params['boosting_type'] = 'gbdt'
params['objective'] = 'regression'
params['metric'] = 'mae'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['min_hessian'] = 1

watchlist = [d_valid]
clf = lgb.train(params, d_train, 500, watchlist)

del d_train, d_valid; gc.collect()
del x_train, x_valid; gc.collect()

[1]	valid_0's l1: 0.0721256
[2]	valid_0's l1: 0.0721274
[3]	valid_0's l1: 0.0721273
[4]	valid_0's l1: 0.0721282
[5]	valid_0's l1: 0.0721287
[6]	valid_0's l1: 0.0721278
[7]	valid_0's l1: 0.0721238
[8]	valid_0's l1: 0.0721234
[9]	valid_0's l1: 0.0721213
[10]	valid_0's l1: 0.0721192
[11]	valid_0's l1: 0.0721198
[12]	valid_0's l1: 0.072119
[13]	valid_0's l1: 0.072119
[14]	valid_0's l1: 0.0721205
[15]	valid_0's l1: 0.0721179
[16]	valid_0's l1: 0.0721174
[17]	valid_0's l1: 0.0721168
[18]	valid_0's l1: 0.0721116
[19]	valid_0's l1: 0.0721145
[20]	valid_0's l1: 0.0721149
[21]	valid_0's l1: 0.0721097
[22]	valid_0's l1: 0.0721073
[23]	valid_0's l1: 0.0721093
[24]	valid_0's l1: 0.0721112
[25]	valid_0's l1: 0.0721079
[26]	valid_0's l1: 0.0721093
[27]	valid_0's l1: 0.0721053
[28]	valid_0's l1: 0.0721058
[29]	valid_0's l1: 0.0721016
[30]	valid_0's l1: 0.0720981
[31]	valid_0's l1: 0.072094
[32]	valid_0's l1: 0.0720943
[33]	valid_0's l1: 0.072093
[34]	valid_0's l1: 0.0720918
[35]	valid_0's l1: 0.072093

[280]	valid_0's l1: 0.0720086
[281]	valid_0's l1: 0.0720095
[282]	valid_0's l1: 0.072009
[283]	valid_0's l1: 0.0720088
[284]	valid_0's l1: 0.0720097
[285]	valid_0's l1: 0.0720089
[286]	valid_0's l1: 0.072009
[287]	valid_0's l1: 0.0720069
[288]	valid_0's l1: 0.0720077
[289]	valid_0's l1: 0.0720056
[290]	valid_0's l1: 0.0720053
[291]	valid_0's l1: 0.0720075
[292]	valid_0's l1: 0.0720053
[293]	valid_0's l1: 0.0720068
[294]	valid_0's l1: 0.0720089
[295]	valid_0's l1: 0.072008
[296]	valid_0's l1: 0.07201
[297]	valid_0's l1: 0.0720091
[298]	valid_0's l1: 0.0720077
[299]	valid_0's l1: 0.0720063
[300]	valid_0's l1: 0.0720066
[301]	valid_0's l1: 0.0720032
[302]	valid_0's l1: 0.0720041
[303]	valid_0's l1: 0.0720014
[304]	valid_0's l1: 0.0720034
[305]	valid_0's l1: 0.0720009
[306]	valid_0's l1: 0.0719965
[307]	valid_0's l1: 0.0719963
[308]	valid_0's l1: 0.0719976
[309]	valid_0's l1: 0.071997
[310]	valid_0's l1: 0.0719985
[311]	valid_0's l1: 0.0719993
[312]	valid_0's l1: 0.0719996
[313]	valid_0's 

0

In [20]:
print("Prepare for the prediction ...")
sample = pd.read_csv('sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')
del sample, prop; gc.collect()
x_test = df_test[train_columns]
del df_test; gc.collect()
for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
x_test = x_test.values.astype(np.float32, copy=False)

Prepare for the prediction ...


In [21]:
print("Start prediction ...")
# num_threads > 1 will predict very slow in kernal
clf.reset_parameter({"num_threads":1})
p_test = clf.predict(x_test)

del x_test; gc.collect()

Start prediction ...


7

In [23]:
print("Start write result ...")
sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('lgb_starter.csv', index=False, float_format='%.4f')

Start write result ...
